This notebook contains the code for cleaning up the crime, weather, demographics  datasets and combining them into one VIEW for later analysis

We connect to AWS postgres Database and send queries 

In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import ast


# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline 



connection_args = {
    'host': '54.185.23.30',  # We are connecting to our _local_ version of psql
    'user': 'ubuntu',
    'dbname': 'crime',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [2]:
query = 'SELECT * FROM crime_2012_2017 LIMIT 5;'

crime_df = pd_sql.read_sql(query, connection)

In [113]:
crime_df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location,Longitude,Latitude,Hour_Occurred,Time_Occurred_with_hour
0,605,2012-06-07,2012-06-06,2300,5,Harbor,518,510,VEHICLE - STOLEN,None,...,NaN,None,None,24600 AVALON BL,None,"{'latitude': '33.8028', 'needs_recoding': Fals...",-118.2640,33.8028,23,2012-06-06 23:00:00
1,110322412,2012-10-06,2012-09-18,1200,3,Southwest,328,662,"BUNCO, GRAND THEFT",0701,...,NaN,None,None,700 W 27TH ST,None,"{'latitude': '34.0278', 'needs_recoding': Fals...",-118.2783,34.0278,12,2012-09-18 12:00:00
2,110923275,2012-01-06,2012-01-06,1530,15,N Hollywood,1512,510,VEHICLE - STOLEN,None,...,NaN,None,None,ALCOVE AV,VOSE ST,"{'latitude': '34.1985', 'needs_recoding': Fals...",-118.4126,34.1985,15,2012-01-06 15:00:00
3,112023635,2012-01-17,2012-01-17,2140,11,Northeast,1143,510,VEHICLE - STOLEN,1402 1309 0916 0342,...,998.0,None,None,3800 TRACY ST,None,"{'latitude': '34.1059', 'needs_recoding': Fals...",-118.2755,34.1059,21,2012-01-17 21:00:00
4,120100001,2012-03-13,2012-03-12,2000,1,Central,101,510,VEHICLE - STOLEN,None,...,NaN,None,None,1200 W SUNSET BL,None,"{'latitude': '34.0682', 'needs_recoding': Fals...",-118.2502,34.0682,20,2012-03-12 20:00:00


In [114]:
# Explore the Date Reported and Occurred columns 
query = 'SELECT "Date Reported", "Date Occurred"  FROM crime_2012_2017  WHERE "Date Reported"<"Date Occurred";'
# There are no columns where the "Date Reported" is earlier than "Date Occurred"

pd_sql.read_sql(query, connection)

,Date Reported,Date Occurred


In [115]:
query = 'SELECT "Date Reported", "DR Number" FROM crime_2012_2017 WHERE "Date Reported" IS NULL;'
# There are no columns where "Date Reported" is missing

pd_sql.read_sql(query, connection)

,Date Reported,DR Number


In [116]:
# Clean up "Area ID" and "Area Name" Columns
query = 'SELECT DISTINCT("Area ID", "Area Name") FROM crime_2012_2017;'
# There are 21 distircts and no mismatch between the district and their names

pd_sql.read_sql(query, connection)


,row
0,"(1,Central)"
1,"(2,Rampart)"
2,"(3,Southwest)"
3,"(4,Hollenbeck)"
4,"(5,Harbor)"
5,"(6,Hollywood)"
6,"(7,Wilshire)"
7,"(8,""West LA"")"
8,"(9,""Van Nuys"")"
9,"(10,""West Valley"")"


In [117]:
query = 'SELECT "Area ID", "Area Name" FROM crime_2012_2017 where "Area ID" IS NULL;'
# No null values for "Area ID"
# Will probably not use the "Area Name" as a feature in this project

pd_sql.read_sql(query, connection)


,Area ID,Area Name


In [118]:
# Check to see if "Reporting District" has any nulls
query = 'SELECT "Reporting District" FROM crime_2012_2017 where "Reporting District" IS NULL;'
#  There are no nulls in the "Reporting District" column

pd_sql.read_sql(query, connection)



,Reporting District


In [119]:
# Clean up "Area ID" and "Area Name" Columns
query = 'SELECT DISTINCT("Crime Code", "Crime Code Description") FROM crime_2012_2017;'
# There are 138 Crime Codes and no mismatch between the code and their description

pd_sql.read_sql(query, connection)


,row
0,"(110,""CRIMINAL HOMICIDE"")"
1,"(113,""MANSLAUGHTER, NEGLIGENT"")"
2,"(121,""RAPE, FORCIBLE"")"
3,"(122,""RAPE, ATTEMPTED"")"
4,"(210,ROBBERY)"
...,...
133,"(950,""DEFRAUDING INNKEEPER/THEFT OF SERVICES, ..."
134,"(951,""DEFRAUDING INNKEEPER/THEFT OF SERVICES, ..."
135,"(952,ABORTION/ILLEGAL)"
136,"(954,CONTRIBUTING)"


In [120]:
query = 'SELECT "Crime Code", "Crime Code Description" FROM crime_2012_2017 where "Crime Code" IS NULL;'
# No null values for "Crime Code"
# Will probably not use the "Crime Code Description" as a feature in this project

pd_sql.read_sql(query, connection)


,Crime Code,Crime Code Description


The next 4 queries are on the victim related columns which are not cleaned, they will be cleaned if decided to use in the analysis

In [121]:
# Checking for Null values in the Victim Age, Sex and Descent columns
query = 'SELECT COUNT("Victim Age") FROM crime_2012_2017 WHERE "Victim Age" IS NOT NULL;'
# No null values in the age column
# 1257800 non null values

pd_sql.read_sql(query, connection)


,count
0,1257800


In [122]:
query = 'SELECT Count("Victim Age") FROM crime_2012_2017 WHERE "Victim Sex" IS NULL;'
# 1140726 non Null values
# 117074 null values for Victim Sex

pd_sql.read_sql(query, connection)


,count
0,117074


In [123]:
query = 'SELECT COUNT("Victim Age") FROM crime_2012_2017 WHERE "Victim Age" <= 0;'
# 217367 non positive "Victim Age"

pd_sql.read_sql(query, connection)


,count
0,217367


In [124]:
query = 'SELECT COUNT("Victim Age") FROM crime_2012_2017 WHERE "Victim Descent" IS NOT NULL;'
# 117103 Null values in Victim Descent column
# 1140697 not null values in the Vic Descent column 

pd_sql.read_sql(query, connection)


,count
0,1140697


In [125]:
# Checking how many ditinct premise codes there are
query = 'SELECT COUNT(DISTINCT("Premise Code")) FROM crime_2012_2017;'
# 305

pd_sql.read_sql(query, connection)



,count
0,305


In [126]:
# Display the distinct "Premise Code"
query = 'SELECT DISTINCT("Premise Code") FROM crime_2012_2017;'
# There are some Null values, let's check below how many there are

pd_sql.read_sql(query, connection)


,Premise Code
0,908.0
1,947.0
2,519.0
3,NaN
4,154.0
...,...
301,209.0
302,918.0
303,725.0
304,504.0


In [127]:
# check out if there are any null values for the premise code column
query = 'SELECT COUNT("Premise Code") FROM crime_2012_2017 WHERE "Premise Code" IS NOT NULL;'
# 1257775 Not null values
# 25 null values in the"Premise Code" column

pd_sql.read_sql(query, connection)


,count
0,1257775


In [128]:
# check to see if the "Premise Desc" is present where the "Premise Code" is null
query = 'SELECT "Premise Code", "Premise Description","Status Code", "Status Description" FROM crime_2012_2017 WHERE "Premise Code" IS NULL;'
# For all 25 null values for "Premise Code" "Premise desc" is also missing
# I might drop those or use the lat and long to find the type

pd_sql.read_sql(query, connection)


,Premise Code,Premise Description,Status Code,Status Description
0,None,None,IC,Invest Cont
1,None,None,CC,UNK
2,None,None,IC,Invest Cont
3,None,None,IC,Invest Cont
4,None,None,AA,Adult Arrest
5,None,None,AA,Adult Arrest
6,None,None,AA,Adult Arrest
7,None,None,IC,Invest Cont
8,None,None,AO,Adult Other
9,None,None,IC,Invest Cont


In [129]:
# Checkout the status code and see the distinct types of the statuses
query = 'SELECT DISTINCT("Status Code") FROM crime_2012_2017;'
# AA,IC,13,CC,TH,19,JA,JO, AO
# There are also some null values, let's check how many and if for those the "Status Desc" is also missing

pd_sql.read_sql(query, connection)

,Status Code
0,None
1,AA
2,IC
3,13
4,CC
5,19
6,TH
7,JA
8,JO
9,AO


In [130]:
query = 'SELECT "Status Code", "Status Description" FROM crime_2012_2017 WHERE "Status Code" IS NULL or "Status Code" = \'13\' or "Status Code" = \'19\' or "Status Code" = \'CC\'or "Status Code" = \'TH\';'
# There is only one entry with missing Status Code and Status Desc = UNK (maybe unknown?)
# We will drop this

pd_sql.read_sql(query, connection)


,Status Code,Status Description
0,TH,UNK
1,CC,UNK
2,CC,UNK
3,CC,UNK
4,None,UNK
5,CC,UNK
6,CC,UNK
7,CC,UNK
8,CC,UNK
9,CC,UNK


In [131]:
query = 'SELECT "Status Code", "Status Description" FROM crime_2012_2017 WHERE  "Status Description" = \'UNK\';'
# There is only one entry with missing Status Code and Status Desc = UNK (maybe unknown?)
# We will drop this

pd_sql.read_sql(query, connection)


,Status Code,Status Description
0,TH,UNK
1,CC,UNK
2,None,UNK
3,CC,UNK
4,CC,UNK
5,CC,UNK
6,CC,UNK
7,CC,UNK
8,CC,UNK
9,CC,UNK


In [134]:
# create a view of the main table so when we make changes to it we don't alter the main data

cur = connection.cursor()

query = 'CREATE VIEW clean_crime AS SELECT * FROM crime_2012_2017;'
cur.execute(query)
connection.commit()



In [136]:
# Deleting the row where status description is 'UNK'. those are 23 rows where the status is not AC,AO, JC, JO

query = 'DELETE FROM clean_crime WHERE "Status Description" = \'UNK\';'
cur.execute(query)
connection.commit()
cur.close()
# this deleted the null status code and some of the weird status coedes that do not have description

In [140]:
# checking to see if delete worked
query = 'SELECT "Status Code", "Status Description" FROM clean_crime WHERE "Status Description" = \'UNK\';'

pd_sql.read_sql(query, connection)


,Status Code,Status Description


In [141]:
# check if delete worked 
query = 'SELECT COUNT(*) FROM clean_crime;'
# 23 less rows

pd_sql.read_sql(query, connection)


,count
0,1257777


Now we will create our target vector --> if arrested then 1, if not arrested then 0

In [142]:
# Now we will create our target vector --> if arrested then 1, if not arrested then 0
# Target vector comes from the "Status Code"
query = 'SELECT "Status Code", "Status Description" FROM clean_crime;'

pd_sql.read_sql(query, connection)


,Status Code,Status Description
0,IC,Invest Cont
1,IC,Invest Cont
2,IC,Invest Cont
3,AO,Adult Other
4,IC,Invest Cont
...,...,...
1257772,IC,Invest Cont
1257773,IC,Invest Cont
1257774,IC,Invest Cont
1257775,AO,Adult Other


In [82]:
query = 'SELECT DISTINCT("Status Code") FROM clean_crime;'
# AA, AO, JA, JO, IC distinct codes left 

pd_sql.read_sql(query, connection)


,Status Code
0,AA
1,IC
2,JA
3,JO
4,AO


In [143]:
query = 'CREATE VIEW target_arrested_vector AS SELECT "DR Number", "Status Code", CASE WHEN "Status Code" = \'AA\' or "Status Code" = \'JA\' THEN 1 ELSE 0 END AS target_arrested FROM clean_crime;'

cur.execute(query)
connection.commit()



In [159]:
# Extract the Month of the "Date Reported"
cur = connection.cursor()
query = """ CREATE VIEW month_and_date_reported ("Month_of_Reported","Date Reported","DR Number") AS 
    SELECT EXTRACT (MONTH FROM "Date Reported"), "Date Reported", 
    "DR Number"  FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [160]:
# EXTRACT the Day of the "Date Reported"
cur = connection.cursor()
query = """CREATE VIEW day_and_date_reported ("Day_of_Reported","Date Reported","DR Number") AS  
        SELECT EXTRACT (Day FROM "Date Reported"),
        "Date Reported", "DR Number" FROM clean_crime;"""

cur.execute(query)
connection.commit()
cut.close()

In [161]:
# EXTRACT the year of the "Date Reported"
cur = connection.cursor()

query = """CREATE VIEW year_and_date_reported ("Year_of_Reported", "Date Reported", "DR Number") AS 
        SELECT EXTRACT (YEAR FROM "Date Reported"), "Date Reported",
        "DR Number"  FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [162]:
# Extract the day of the week from "Date Reported"
# --> Sunday = 0, ..., Saturday == 6
cur = connection.cursor()

query = """CREATE VIEW dow_and_date_reported (day_of_week_of_Reported, "Date Reported", "DR Number")
            AS SELECT EXTRACT(dow from "Date Reported"),
            "Date Reported", "DR Number" FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [173]:
# Extract the Month of the "Date Occurred"

cur = connection.cursor()

query = """ CREATE VIEW month_and_date_occurred ("Month_of_Occurred","Date Occurred","DR Number") AS 
    SELECT EXTRACT (MONTH FROM "Date Occurred"), "Date Occurred", 
    "DR Number"  FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [174]:
# EXTRACT the Day of the "Date Occurred"
cur = connection.cursor()

query = """CREATE VIEW day_and_date_occurred ("Day_of_Occurred","Date Occurred","DR Number") AS  
        SELECT EXTRACT (Day FROM "Date Occurred"),
        "Date Occurred", "DR Number" FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [175]:
# EXTRACT the year of the "Date Occurred"
cur = connection.cursor()

query = """CREATE VIEW year_and_date_occurred ("Year_of_Occurred", "Date Occurred", "DR Number") AS 
        SELECT EXTRACT (YEAR FROM "Date Occurred"), "Date Occurred",
        "DR Number"  FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

In [176]:
# Extract the day of the week from "Date Occurred"
# --> Sunday = 0, ..., Saturday == 6
cur = connection.cursor()
query = """CREATE VIEW dow_and_date_occurred (day_of_week_of_Occurred, "Date Occurred", "DR Number")
            AS SELECT EXTRACT(dow from "Date Occurred"),
            "Date Occurred", "DR Number" FROM clean_crime;"""

cur.execute(query)
connection.commit()
cur.close()

We will now change the weather related tables to add to clean_crime
* humidity
* pressure (probably won't use)
* temperature
* wind_speed 
* wind_direction (won't use)
* weather_desc (won't use)

In [9]:
cur = connection.cursor()
query = '''CREATE VIEW la_temperature ("Date_Time", "LA_temp")
         AS SELECT "datetime", "Los Angeles" FROM temperature;'''
cur.execute(query)
connection.commit()
cur.close()

In [10]:
# temperature is in kelvin, switching to celsius
cur = connection.cursor()
query = 'UPDATE la_temperature SET "LA_temp" = "LA_temp" - 273.15;'
cur.execute(query)
connection.commit()
cur.close()


In [11]:
# create a wind speed view
cur = connection.cursor()
query = '''CREATE VIEW la_wind_speed ("Date_Time", "LA_wind_speed")
         AS SELECT "datetime", "Los Angeles" FROM wind_speed;'''
cur.execute(query)
connection.commit()
cur.close()

In [12]:
# create a humidity view
cur = connection.cursor()
query = '''CREATE VIEW la_humidity ("Date_Time", "LA_humidity")
         AS SELECT "datetime", "Los Angeles" FROM humidity;'''
cur.execute(query)
connection.commit()
cur.close()

Now let's look at the demographics information

In [233]:
query = 'SELECT * FROM census_council_district;'
council_df = pd_sql.read_sql(query, connection)

In [237]:
council_df

,Council District,Pop2010,White_pop,Black_pop,Ameri_es_pop,Asian_pop,Hawn_pi_pop,Hispanic_pop,Other_pop,Multi_pop,...,Marhh_chd,Marhh_no_c,Mhh_child,Fhh_child,Families,Avg_family_size,Housing_units,Vacant,Owner_occ,Renter_occ
0,1 - Gilbert Cedillo,245216.22,90857.00,7759.13,2991.73,43397.31,243.41,172971.55,88016.11,11951.53,...,17472.38,12582.79,3923.03,8697.64,51391.85,2.68,85219.33,6204.52,15219.58,63795.23
1,10 - Herb J. Wesson Jr.,261297.85,68458.12,67087.07,2307.88,43148.34,308.11,126872.43,68188.27,11800.07,...,17301.48,14284.08,3736.82,10927.24,57677.78,2.52,103637.46,7721.54,21764.30,74151.62
2,11 - Mike Bonin,250726.93,174020.70,13569.60,1214.43,30132.14,521.02,47243.32,18894.80,12374.24,...,17305.63,24708.30,1874.20,4632.45,55447.15,2.07,123048.13,9183.91,50753.74,63110.49
3,12 - Mitchell Englander,258715.95,158939.64,11239.45,1193.56,46815.27,374.43,70827.63,27943.83,12209.77,...,22493.92,26040.98,2137.22,5223.44,63921.59,2.72,90415.80,3683.73,60274.21,26457.86
4,13 - Mitch O'Farrell,247142.04,113391.23,9109.58,2320.07,43740.54,315.72,133038.67,66020.53,12244.39,...,15096.73,14479.35,3100.50,7569.90,49457.04,2.48,102632.78,7725.95,13638.56,81268.27
5,14 - Jose Huizar,248489.10,113548.42,15693.05,2816.77,31591.55,243.26,167529.97,74188.21,10407.84,...,16047.25,13208.00,3017.05,7495.71,47273.89,2.60,85992.89,6787.23,26212.29,52993.37
6,15 - Joe Buscaino,251283.55,108150.70,32853.03,2213.40,16305.70,1372.82,156217.43,77268.88,13119.01,...,19513.33,13730.91,3716.83,11151.98,55863.03,2.55,81562.44,5136.14,31668.19,44758.11
7,2 - Paul Krekorian,256691.84,158999.91,11463.05,1526.06,17554.92,330.51,115511.82,54814.59,12002.80,...,19249.41,17423.32,3124.50,7409.10,55314.34,2.57,101135.77,6337.16,33612.25,61186.36
8,3 - Bob Blumenfield,257183.29,155558.89,11669.04,1430.11,30404.92,324.33,95591.32,44526.13,13269.87,...,21684.98,21069.27,2985.33,6800.72,60784.85,2.74,95650.77,6171.92,46723.56,42755.29
9,4 - David Ryu,244117.22,169640.22,12524.23,730.64,36798.67,225.96,36760.90,14169.79,10027.72,...,15361.81,21752.67,1844.77,4538.83,50875.22,1.75,130367.28,9377.20,42201.31,78788.78


In [9]:
# we will update the clean_crime view by adding a council 
# district column to be able to put the 2 df together
cur = connection.cursor()

cur.execute("""CREATE VIEW area_council_codes AS
            SELECT "DR Number", "Area ID",  
            CASE WHEN "Area ID" = 1 THEN 14
            WHEN "Area ID" = 2 THEN 13
            WHEN "Area ID" = 3 THEN 10
            WHEN "Area ID" = 4 THEN 14
            WHEN "Area ID" = 5 THEN 15
            WHEN "Area ID" = 6 THEN 4
            WHEN "Area ID" = 7 THEN 4
            WHEN "Area ID" = 8 THEN 11
            WHEN "Area ID" = 9 THEN 6
            WHEN "Area ID" = 10 THEN 5
            WHEN "Area ID" = 11 THEN 1
            WHEN "Area ID" = 12 THEN 8
            WHEN "Area ID" = 13 THEN 9
            WHEN "Area ID" = 14 THEN 11
            WHEN "Area ID" = 15 THEN 2
            WHEN "Area ID" = 16 THEN 7
            WHEN "Area ID" = 17 THEN 12
            WHEN "Area ID" = 18 THEN 15
            WHEN "Area ID" = 19 THEN 7
            WHEN "Area ID" = 20 THEN 13
            WHEN "Area ID" = 21 THEN 3
            END AS "Council Code"
            FROM clean_crime;""")

connection.commit()
cur.close()

In [54]:
# spliting the council code from the council name to be able to attach 
# the area_council_codes that contains "Area ID" 
cur = connection.cursor()

query = '''CREATE VIEW council_district_updated_code_column AS
SELECT SPLIT_PART("Council District", ' - ', 1) council_code, 
SPLIT_PART("Council District", ' - ', 2) council_name,
census_council_district.*
FROM census_council_district
;
'''
cur.execute(query)
connection.commit()
cur.close()


In [2]:
# changing the columntype of the above view t int to be able
# to combine with another view
cur = connection.cursor()

query = '''CREATE VIEW council_district_updated_code_integer_column AS 
SELECT 
CASE
WHEN "council_code" = '1' THEN 1
WHEN "council_code" = '2' THEN 2
WHEN "council_code" = '3' THEN 3
WHEN "council_code" = '4' THEN 4
WHEN "council_code" = '5' THEN 5
WHEN "council_code" = '6' THEN 6
WHEN "council_code" = '7' THEN 7
WHEN "council_code" = '8' THEN 8
WHEN "council_code" = '9' THEN 9
WHEN "council_code" = '10' THEN 10
WHEN "council_code" = '11' THEN 11
WHEN "council_code" = '12' THEN 12
WHEN "council_code" = '13' THEN 13
WHEN "council_code" = '14' THEN 14
WHEN "council_code" = '15' THEN 15
END AS "council_code_integer",
council_district_updated_code_column.*
FROM council_district_updated_code_column;'''
cur.execute(query)
connection.commit()
cur.close()

In [6]:
# putting the area id, dr num and council distirct info all together under one view
cur = connection.cursor()

query = '''CREATE VIEW council_district_with_area_id_and_dr_num AS
SELECT area_council_codes.*, council_district_updated_code_integer_column.*
FROM area_council_codes
LEFT JOIN council_district_updated_code_integer_column ON 
council_district_updated_code_integer_column.council_code_integer = area_council_codes."Council Code"

'''
cur.execute(query)
connection.commit()
cur.close()


Join all the views into one final view 

In [27]:
cur = connection.cursor()

query = """CREATE VIEW new_clean_crime AS

SELECT clean_crime.*, target_arrested_vector."target_arrested",
year_and_date_occurred."Year_of_Occurred", year_and_date_reported."Year_of_Reported", 
month_and_date_occurred."Month_of_Occurred",month_and_date_reported."Month_of_Reported",
day_and_date_occurred."Day_of_Occurred", day_and_date_reported."Day_of_Reported", 
dow_and_date_occurred."day_of_week_of_occurred",dow_and_date_reported."day_of_week_of_reported",
la_humidity."LA_humidity", la_temperature."LA_temp", la_wind_speed."LA_wind_speed",

FROM clean_crime

LEFT JOIN target_arrested_vector ON target_arrested_vector."DR Number" = clean_crime."DR Number"
LEFT JOIN year_and_date_reported ON year_and_date_reported."DR Number" = clean_crime."DR Number"
LEFT JOIN month_and_date_reported ON month_and_date_reported."DR Number" = clean_crime."DR Number"
LEFT JOIN day_and_date_reported ON day_and_date_reported."DR Number" = clean_crime."DR Number"
LEFT JOIN dow_and_date_reported ON dow_and_date_reported."DR Number" = clean_crime."DR Number"
LEFT JOIN year_and_date_occurred ON year_and_date_occurred."DR Number" = clean_crime."DR Number"
LEFT JOIN month_and_date_occurred ON month_and_date_occurred."DR Number" = clean_crime."DR Number"
LEFT JOIN day_and_date_occurred ON day_and_date_occurred."DR Number" = clean_crime."DR Number"
LEFT JOIN dow_and_date_occurred ON dow_and_date_occurred."DR Number" = clean_crime."DR Number"
LEFT JOIN la_humidity ON la_humidity."Date_Time" = clean_crime."Time_Occurred_with_hour"
LEFT JOIN la_temperature ON la_temperature."Date_Time" = clean_crime."Time_Occurred_with_hour"
LEFT JOIN la_wind_speed ON la_wind_speed."Date_Time" = clean_crime."Time_Occurred_with_hour"
;"""

# pd_sql.read_sql(query, connection)
cur.execute(query)
connection.commit()
cur.close()

In [10]:
cur = connection.cursor()

query = '''CREATE VIEW final_crime_and_council_data AS
SELECT 
new_clean_crime."DR Number", 
new_clean_crime."Date Reported",
new_clean_crime."Date Occurred",
new_clean_crime."Area ID",
new_clean_crime."Area Name",
new_clean_crime."Reporting District",
new_clean_crime."Crime Code",
new_clean_crime."Crime Code Description",
new_clean_crime."Premise Code",
new_clean_crime."Premise Description",
new_clean_crime."Weapon Used Code",
new_clean_crime."Latitude",
new_clean_crime."Longitude",
new_clean_crime."Hour_Occurred",
new_clean_crime."Time_Occurred_with_hour",
new_clean_crime."target_arrested",
new_clean_crime."Year_of_Occurred",
new_clean_crime."Year_of_Reported",
new_clean_crime."Month_of_Occurred",
new_clean_crime."Month_of_Reported",
new_clean_crime."Day_of_Occurred",
new_clean_crime."Day_of_Reported",
new_clean_crime."day_of_week_of_occurred",
new_clean_crime."day_of_week_of_reported",
new_clean_crime."LA_temp",
council_district_with_area_id_and_dr_num."council_code_integer",
council_district_with_area_id_and_dr_num."Pop2010",
council_district_with_area_id_and_dr_num."White_pop",
council_district_with_area_id_and_dr_num."Black_pop",
council_district_with_area_id_and_dr_num."Ameri_es_pop",
council_district_with_area_id_and_dr_num."Asian_pop",
council_district_with_area_id_and_dr_num."Hawn_pi_pop",
council_district_with_area_id_and_dr_num."Hispanic_pop",
council_district_with_area_id_and_dr_num."Other_pop",
council_district_with_area_id_and_dr_num."Multi_pop",
council_district_with_area_id_and_dr_num."Male_pop",
council_district_with_area_id_and_dr_num."Female_pop",
council_district_with_area_id_and_dr_num."Median_age",
council_district_with_area_id_and_dr_num."Med_age_male",
council_district_with_area_id_and_dr_num."Med_age_female",
council_district_with_area_id_and_dr_num."Avg_hh_size",
council_district_with_area_id_and_dr_num."Vacant",
council_district_with_area_id_and_dr_num."Owner_occ",
council_district_with_area_id_and_dr_num."Renter_occ"

FROM new_clean_crime
LEFT JOIN council_district_with_area_id_and_dr_num ON 
council_district_with_area_id_and_dr_num."DR Number" = new_clean_crime."DR Number" ;
'''
cur.execute(query)
connection.commit()
cur.close()


In [4]:
#cur = connection.cursor()
query = 'SELECT "DR Number", "Victim Age", "Victim Descent" FROM new_clean_crime;'

victim_df = pd_sql.read_sql(query, connection)

#cur.execute(query)
#connection.commit()
#cur.close()

In [5]:
victim_df.to_csv('crime_victim_df.csv')

If time allows will add the median houseing listing price from zillow

Load the dataset into pandas

In [ ]:
crime_df = pd_sql.read_sql('SELECt * FROM final_crime_and_council_data;', connection)

In [7]:
victim_crime_df = pd.merge(crime_df,victim_df, on = 'DR Number')

In [8]:
victim_crime_df.to_csv('Victim_Crime.csv')

In [16]:
crime_victim_df.tail()

,Unnamed: 0,DR Number,Date Reported,Date Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,Premise Code,...,Med_age_male,Med_age_female,Avg_hh_size,Vacant,Owner_occ,Renter_occ,Unnamed: 0.1,Victim Age,Victim Descent,Crime_Code_Name
1099995,120626,152109415,2015-04-30 00:00:00,2015-04-30 00:00:00,21,Topanga,2129,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",501.0,...,29.75,31.71,2.38,6171.92,46723.56,42755.29,74762,23,O,Agg.Assault
1099996,120627,152110024,2015-05-13 00:00:00,2015-05-13 00:00:00,21,Topanga,2105,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),108.0,...,29.75,31.71,2.38,6171.92,46723.56,42755.29,74774,70,O,SimpleAssault
1099997,120628,152110574,2015-05-26 00:00:00,2015-05-20 00:00:00,21,Topanga,2146,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",721.0,...,29.75,31.71,2.38,6171.92,46723.56,42755.29,74777,35,X,Other_Theft
1099998,120629,152110854,2015-05-31 00:00:00,2015-05-29 00:00:00,21,Topanga,2107,330,BURGLARY FROM VEHICLE,101.0,...,29.75,31.71,2.38,6171.92,46723.56,42755.29,74782,32,A,BTFV
1099999,120630,152111150,2015-06-06 00:00:00,2015-06-04 00:00:00,21,Topanga,2106,662,"BUNCO, GRAND THEFT",502.0,...,29.75,31.71,2.38,6171.92,46723.56,42755.29,74786,88,W,Other
